# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from time import time

%matplotlib inline
%matplotlib notebook
plt.style.use('default')

# Operaciones con los dataframes

### DF 1: Educación de los postulantes

Realizamos un merge de los 3 dataframes de educación de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [2]:
postulantes_educ = pd.read_csv('datos_navent_fiuba/h15_fiuba_1_postulantes_educacion.csv')
postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, how='outer')

postulantes_educ_2 = pd.read_csv('datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

postulantes_educ = pd.merge(postulantes_educ, postulantes_educ_2, how='outer')

# Renombramos la columna 'nombre' por algo mas apropiado
postulantes_educ.rename(columns={'nombre':'titulo_univ'}, inplace=True)

postulantes_educ.head()

,idpostulante,titulo_univ,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [3]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 693641 entries, 0 to 693640
Data columns (total 3 columns):
idpostulante    693641 non-null object
titulo_univ     693641 non-null object
estado          693641 non-null object
dtypes: object(3)
memory usage: 21.2+ MB


Transformamos los strings de los títulos universitarios en variables numéricas.
En principio esto lo realizamos para poder eliminar los IDs duplicados del dataframe. Aquí se contemplan los casos de postulantes que ingresaron con distintos títulos o que el estado de su título varió entre una postulación y la otra. A su vez, esto nos servirá más adelante para aplicar los algoritmos de Machine Learning.

Analizamos los distintos tipos de títulos universitarios que existen.

In [4]:
postulantes_educ.groupby('titulo_univ')['titulo_univ'].count()

titulo_univ
Doctorado               620
Master                10074
Otro                  53363
Posgrado              20624
Secundario           244689
Terciario/Técnico    113421
Universitario        250850
Name: titulo_univ, dtype: int64

Pasamos a variables numéricas cada uno de los posibles títulos, a fin de armar un orden de jerarquía de títulos, de manera tal de quedarnos por cada ID del postulante con su título más avanzado.

NOTA: consideramos que 'Otro' es el nivel jerárquico más bajo.

In [5]:
# Otro
postulantes_educ.loc[(postulantes_educ["titulo_univ"] == "Otro"), 'estudios'] = 1
 
# Secundario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Secundario'), 'estudios'] = 2

# Terciario/ Técnico
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Terciario/Técnico'), 'estudios'] = 3

# Universitario
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Universitario'), 'estudios'] = 4

# Posgrado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Posgrado'), 'estudios'] = 5

# Master
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Master'), 'estudios'] = 6

# Doctorado
postulantes_educ.loc[(postulantes_educ['titulo_univ'] == 'Doctorado'), 'estudios'] = 7

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
644654,RzPm1w1,Otro,En Curso,1.0
138247,5mPoE0p,Secundario,Graduado,2.0
199682,Z3q9M8,Universitario,Graduado,4.0
101449,LNdlZ0j,Posgrado,Graduado,5.0
458283,qe2b8BX,Secundario,Graduado,2.0


Analizamos la cantidad de IDs duplicados

In [6]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

180088

Ordenamos el dataframe según el orden jerárquico de estudio en forma descendente.

In [7]:
postulantes_educ = postulantes_educ.sort_values('estudios', ascending=False)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por orden de estudio, quedará un único ID por postulante con su título más avanzado.

In [8]:
postulantes_educ = postulantes_educ.drop_duplicates('idpostulante')
postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios
551010,JBxDVzJ,Universitario,En Curso,4.0
473106,6PB65r,Universitario,Graduado,4.0
573243,YmYBR1,Secundario,Graduado,2.0
476521,mzdDE2O,Terciario/Técnico,Graduado,3.0
446242,ekOvRP4,Universitario,En Curso,4.0


Comprobamos que efectivamente no quedaron IDs duplicados.

In [9]:
IDs_duplicados = sum(postulantes_educ['idpostulante'].value_counts() > 1)
IDs_duplicados

0

Creamos una nueva columna que indica si el postulante en cuestión tiene un título en curso o ya se graduó/ abandonó

In [10]:
postulantes_educ['estado'].value_counts()

Graduado      253166
En Curso      148362
Abandonado     46381
Name: estado, dtype: int64

In [11]:
postulantes_educ.loc[(postulantes_educ['estado'] == 'Graduado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'Abandonado'), 'esta_estudiando'] = 0
postulantes_educ.loc[(postulantes_educ['estado'] == 'En Curso'), 'esta_estudiando'] = 1

postulantes_educ.sample(5)

,idpostulante,titulo_univ,estado,estudios,esta_estudiando
394526,ae8rmW,Terciario/Técnico,Graduado,3.0,0.0
51526,LN8reek,Universitario,Abandonado,4.0,0.0
594242,NzMvJzD,Universitario,En Curso,4.0,1.0
377311,KBrDxBX,Secundario,En Curso,2.0,1.0
257252,8zKV1D,Secundario,Graduado,2.0,0.0


Eliminamos las columnas 'titulo_univ' y 'estado', ya que la información relativa a esto ya se encuentra contenida en 'estudios' y en 'esta_estudiando'

In [12]:
postulantes_educ.drop(columns={'titulo_univ', 'estado'}, axis=1, inplace=True)
postulantes_educ.sample(5)

,idpostulante,estudios,esta_estudiando
325552,JBmejrk,2.0,0.0
436725,YjQND3V,4.0,1.0
330395,LNebVEq,6.0,1.0
372380,GNZvepb,2.0,0.0
246265,6rPBa4l,2.0,0.0


In [13]:
postulantes_educ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 393408 to 530578
Data columns (total 3 columns):
idpostulante       447909 non-null object
estudios           447909 non-null float64
esta_estudiando    447909 non-null float64
dtypes: float64(2), object(1)
memory usage: 13.7+ MB


### DF 2: Nacimiento y sexo de los postulantes

Realizamos un merge de los 3 dataframes de nacimiento y sexo de los postulantes que nos fueron provistos, para poder trabajar de manera unificada.

In [14]:
postulantes_gen_nac = pd.read_csv('datos_navent_fiuba/h15_fiuba_2_postulantes_genero_y_edad.csv')
postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac_2 = pd.read_csv('datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

postulantes_gen_nac = pd.merge(postulantes_gen_nac, postulantes_gen_nac_2, how='outer')

postulantes_gen_nac.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [15]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 505381
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    478699 non-null object
sexo               505382 non-null object
dtypes: object(3)
memory usage: 15.4+ MB


Analizamos los posibles valores de la columna 'sexo'

In [16]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
0.0                7
Name: sexo, dtype: int64

Descartamos los registros que figuran con 'sexo' == 0.0, ya que son una pequeñísima parte del set de datos y no brindan información consistente.

In [17]:
postulantes_gen_nac = postulantes_gen_nac[postulantes_gen_nac['sexo'] != '0.0']

Comprobamos que efectivamente fueron eliminados esos registros

In [18]:
postulantes_gen_nac['sexo'].value_counts()

FEM           251431
MASC          228008
NO_DECLARA     25936
Name: sexo, dtype: int64

Analizamos la cantidad de IDs duplicados

In [19]:
IDs_duplicados = sum(postulantes_gen_nac['idpostulante'].value_counts() > 1)
IDs_duplicados

972

Ordenamos el dataframe según el sexo en forma ascendente.

In [20]:
postulantes_gen_nac = postulantes_gen_nac.sort_values('sexo', ascending=True)

Eliminamos todos los IDs duplicados. Considerando que previamente se ordenó el dataframe por sexo y que 'NO_DECLARA' tiene un orden alfabético menor que 'FEM' y 'MASC', quedará un único ID por postulante con su información más específica de sexo.

Interpretamos que la información concreta del sexo nos será relevante en posteriores análisis, por eso filtramos los IDs duplicados de esta forma.

In [21]:
postulantes_gen_nac = postulantes_gen_nac.drop_duplicates('idpostulante')
postulantes_gen_nac.sample(5)

,idpostulante,fechanacimiento,sexo
273548,mzd8p93,1979-07-07,FEM
341392,Yj536OJ,1997-09-27,FEM
177159,kPLYLGW,1994-02-08,MASC
238869,akjvmB5,1998-06-12,FEM
176031,0zkwmQa,1990-04-17,MASC


Se transforma la columna 'fechanacimiento' en 'edad', mediante una serie de operaciones lógicas.

In [22]:
# Renombramos la columna por algo más apropiado
postulantes_gen_nac = postulantes_gen_nac.rename(columns={'fechanacimiento': 'edad'})

# Obtenemos el año de nacimiento
fecha_nac = postulantes_gen_nac['edad'].str.split('-')
año_nac = fecha_nac.str[0]
postulantes_gen_nac['edad'] = año_nac
postulantes_gen_nac['edad'] = pd.to_numeric(postulantes_gen_nac['edad'], errors='coerce').fillna(0).astype(np.int64)

# Filtramos edades que consideramos inválidas
edad_min = postulantes_gen_nac['edad'] > 2000
edad_max = postulantes_gen_nac['edad'] < 1950
condicion_final = ((edad_min | edad_max))
postulantes_gen_nac = postulantes_gen_nac[np.logical_not(condicion_final)]

# Obtenemos la edad real según el año actual
postulantes_gen_nac['edad'] = postulantes_gen_nac['edad'].apply(lambda x: 2018-x)

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo
141065,vVe0KrP,21,FEM
257097,ow2XNN3,19,FEM
328341,ow2VeB0,22,FEM
276266,VNrjz1j,25,MASC
448987,6rQm9wx,43,FEM


Creamos la columna rango_edad para minimizar la cantidad de valores posibles correspondientes a la edad del postulante.

Rangos:
- 18-25
- 26-30
- 31-40
- 41-68

In [23]:
rango_18_25 = (postulantes_gen_nac['edad'] >= 18) & (postulantes_gen_nac['edad'] <= 25)
rango_26_30 = (postulantes_gen_nac['edad'] >= 26) & (postulantes_gen_nac['edad'] <= 30)
rango_31_40 = (postulantes_gen_nac['edad'] >= 31) & (postulantes_gen_nac['edad'] <= 40)
rango_41_68 = (postulantes_gen_nac['edad'] >= 41) & (postulantes_gen_nac['edad'] <= 68)

postulantes_gen_nac.loc[(rango_18_25), 'rango_edad'] = 1
postulantes_gen_nac.loc[(rango_26_30), 'rango_edad'] = 2
postulantes_gen_nac.loc[(rango_31_40), 'rango_edad'] = 3
postulantes_gen_nac.loc[(rango_41_68), 'rango_edad'] = 4

postulantes_gen_nac.sample(5)

,idpostulante,edad,sexo,rango_edad
503074,QNrzKEW,20,MASC,1.0
193334,JBxYl9p,21,FEM,1.0
399507,2zP9bNv,19,MASC,1.0
110779,2zmZBE1,26,FEM,2.0
217048,W9WrKvR,20,FEM,1.0


Eliminamos la columna edad. Consideramos que la información importante se encuentra contenida en el rango edad.

In [24]:
postulantes_gen_nac.drop(columns={'edad'}, axis=1, inplace=True)
postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
412922,xkeDG3Y,MASC,2.0
49111,NQZZl4,FEM,2.0
474636,Rz9ARmR,MASC,1.0
18052,1kLW1N,FEM,4.0
309559,6BQoev,MASC,4.0


Analizamos los valores posibles de la columna 'sexo'

In [25]:
postulantes_gen_nac["sexo"].value_counts()

FEM           250194
MASC          226713
NO_DECLARA      1094
Name: sexo, dtype: int64

Pasamos a valor numérico el sexo de los postulantes.

- NO_DECLARA = 0
- FEM = 1
- MASC = 2

In [26]:
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'NO_DECLARA'), 'sexo'] = 0
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'FEM'),'sexo'] = 1
postulantes_gen_nac.loc[(postulantes_gen_nac['sexo'] == 'MASC'),'sexo'] = 2

postulantes_gen_nac.sample(5)

,idpostulante,sexo,rango_edad
212232,JBreRmE,1,1.0
191439,ek4QWDp,1,2.0
432868,Z9o9wZ,1,2.0
10659,evJJRp,2,3.0
314626,6wKqWL,1,3.0


In [27]:
postulantes_gen_nac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478001 entries, 165770 to 147639
Data columns (total 3 columns):
idpostulante    478001 non-null object
sexo            478001 non-null int64
rango_edad      478001 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 14.6+ MB


Liberamos memoria para mejorar la performance

In [28]:
postulantes_educ_2 = 0
postulantes_gen_nac_2 = 0
fecha_nac = 0
año_nac = 0
edad_min = 0
edad_max = 0
condicion_final = 0
rango_18_25 = 0
rango_26_30 = 0
rango_31_40 = 0
rango_41_68 = 0

### Merge de los DF 1 y 2

Se realiza un merge de los dos dataframes con información correspondiente a los postulantes

In [29]:
postulantes = pd.merge(postulantes_educ, postulantes_gen_nac, on='idpostulante', how='inner')

postulantes.head()

,idpostulante,estudios,esta_estudiando,sexo,rango_edad
0,a0XaWD,7.0,0.0,2,4.0
1,4reGo5z,7.0,0.0,1,2.0
2,X9lpKkb,7.0,0.0,2,4.0
3,RzMXJ4E,7.0,0.0,1,4.0
4,akOA6b9,7.0,0.0,2,4.0


In [30]:
postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438451 entries, 0 to 438450
Data columns (total 5 columns):
idpostulante       438451 non-null object
estudios           438451 non-null float64
esta_estudiando    438451 non-null float64
sexo               438451 non-null int64
rango_edad         438451 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 20.1+ MB


### DF 3: Postulaciones a los avisos

Realizamos un merge de los 2 dataframes de postulaciones a los anuncios que nos fueron provistos, para poder trabajar de manera unificada.

In [31]:
postulaciones = pd.read_csv('datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('datos_navent_fiuba/h15_fiuba_4_postulaciones.csv')

postulaciones = pd.merge(postulaciones, postulaciones_2, how='outer')

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


Analizamos la cantidad de registros con los que cuenta el dataframe de postulaciones

In [32]:
registros_totales = len(postulaciones)
registros_totales

6604534

Analizamos la cantidad de postulaciones únicas (de un mismo postulante a un mismo aviso)

In [33]:
registros_unicos = len(postulaciones.drop_duplicates(['idaviso','idpostulante']))
registros_unicos

6603752

In [34]:
registros_duplicados = registros_totales - registros_unicos
registros_duplicados

782

Considerando que hay una diferencia de tan sólo 782 registros (es decir, hay 782 postulaciones duplicadas de un mismo postulante a un mismo aviso), decidimos eliminar los duplicados

In [35]:
postulaciones.drop_duplicates(['idaviso','idpostulante'], inplace=True)

len(postulaciones)

6603752

Eliminamos la columna 'fechapostulacion' ya que no la consideramos relevante para los algoritmos de ML

In [36]:
postulaciones.drop(columns={'fechapostulacion'}, inplace = True)

postulaciones.sample(5)

,idaviso,idpostulante
5097390,1112455614,mzb8pb9
3971093,1112444146,owzORjj
4709918,1112443823,vVjMxrj
2745795,1111960330,ow2dMP0
1721610,1112308156,EzpRLZ0


Analizamos los avisos top (aquellos que reciben mayor cantidad de postulaciones

In [37]:
cant_postulantes_por_aviso = postulaciones.groupby(['idaviso',]).size().to_frame('cant_postulaciones').reset_index()
cant_postulantes_por_aviso.sort_values('cant_postulaciones', inplace = True, ascending = False)

cant_postulantes_por_aviso.head(10)

,idaviso,cant_postulaciones
10405,1112334791,24420
1084,1112094756,22424
931,1112033906,20570
10402,1112334788,16687
1640,1112196813,13619
6977,1112280937,12124
447,1111753681,11102
1837,1112204682,10098
11224,1112345900,9784
766,1111970596,9746


Agregamos la columna 'sepostulo_x' con valor igual 1 para todos los registros. Esto nos será de gran utilidad a la hora de crear el set de entrenamiento para los algoritmos de ML

In [38]:
postulaciones['sepostulo_x'] = 1

postulaciones.sample(10)

,idaviso,idpostulante,sepostulo_x
6341062,1112416123,jk6Okb6,1
2458039,1112297566,0zkRLmr,1
53937,1112322815,MVOqPzv,1
3680282,1112448587,E9dB30,1
93635,1112266297,ak5xvLE,1
3469854,1112399604,6ANxdM,1
1154286,1112281847,N1pBLB,1
690666,1112316042,kPj5mlJ,1
2422318,1112283643,QNMBBG2,1
3666475,1112366913,KBrbaN2,1


### DF 4: Detalles de los avisos

Se realiza un merge de los 3 dataframes de detalles de los avisos que nos fueron provistos

In [39]:
avisos_detalles = pd.read_csv('datos_navent_fiuba/h15_fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/d15_fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles_2 = pd.read_csv('datos_navent_fiuba/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_2, how='outer')

avisos_detalles.sample(5)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
1650,1112216949,1,Scrum Master GBA Norte,<p>Estamos en búsqueda de un Scrum Master para...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Tecnologia / Sistemas,Aliantec
7609,1112238957,1,Analista de Facturación con SAP - PRESENTARSE,"<p><span style=""text-decoration: underline;"">P...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Facturación,Complement Group (holding)
11551,1112253623,1,Operador Senior de Mesa de Ayuda,<p>Para importante medio digital requerimos p...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Tecnologías de la Información,HuCap
22139,1112493855,1,Rectificador Universal Cilíndrico,"<p> </p><p> </p><p><span style="""">Nos encontr...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,IN HAUS
17949,1112341165,1,Administrador de Networking,"<p style=""""><span style="""">IT Resources </span...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Tecnologia / Sistemas,IT Resources


In [40]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25901 entries, 0 to 25900
Data columns (total 11 columns):
idaviso                 25901 non-null int64
idpais                  25901 non-null int64
titulo                  25901 non-null object
descripcion             25901 non-null object
nombre_zona             25901 non-null object
ciudad                  165 non-null object
mapacalle               2125 non-null object
tipo_de_trabajo         25901 non-null object
nivel_laboral           25566 non-null object
nombre_area             25901 non-null object
denominacion_empresa    25894 non-null object
dtypes: int64(2), object(9)
memory usage: 2.4+ MB


Eliminamos los avisos duplicados

In [41]:
avisos_detalles = avisos_detalles.drop_duplicates(['idaviso'])

Eliminamos las columnas que consideramos no relevantes. Según la explicación aquí abajo, éstas seran 'idpais', 'ciudad', 'mapacalle', 'titulo' y 'denominacion_empresa'

In [42]:
# Analizamos la cantidad de 'idpais' posibles
# Dado que hay un único idpais, eliminaremos esta columna.
avisos_detalles['idpais'].value_counts()

1    25288
Name: idpais, dtype: int64

In [43]:
# Analizamos la cantidad de valores nulos en 'ciudad'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['ciudad'].isnull().value_counts()

True     25128
False      160
Name: ciudad, dtype: int64

In [44]:
# Analizamos la cantidad de valores nulos en 'mapacalle'
# Dado el gran porcentaje de valores nulos, eliminaremos la columna.
avisos_detalles['mapacalle'].isnull().value_counts()

True     23219
False     2069
Name: mapacalle, dtype: int64

In [45]:
# Eliminamos las columnas arriba mencionadas
avisos_detalles.drop(columns={'idpais', 'ciudad', 'mapacalle', 'titulo', 'denominacion_empresa'}, axis=1, inplace=True)

avisos_detalles.sample(5)

,idaviso,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
25838,1112457694,<p>En <strong>Serh Consultores</strong> buscam...,Capital Federal,Full-time,NaN,Atención al Cliente
25724,1112428059,<p>Venta de seguros</p><p><u>Coordinador de Se...,Capital Federal,Full-time,NaN,Atención al Cliente
20113,1111293593,"<p style="""">Importante empresa con trayectoria...",Capital Federal,Full-time,Junior,Atención al Cliente
22044,1112492830,<p><strong>Adecco Office</strong> está especia...,Gran Buenos Aires,Full-time,Senior / Semi-Senior,Secretaria
24156,1111749806,<p><strong>Adecco Industrial </strong>está esp...,Gran Buenos Aires,Full-time,Junior,Mantenimiento


Pasamos a valor numérico el nombre zona, según un orden jerárquico asignado por nosotros. Entendemos que Gran Buenos Aires lidera el orden jerárquico, considerando la gran cantidad de registros con los que cuenta. Luego lo siguen Capital Federal y Otros (todas las zonas restantes)

In [46]:
avisos_detalles['nombre_zona'].value_counts()

Gran Buenos Aires              23018
Capital Federal                 2182
Buenos Aires (fuera de GBA)       57
GBA Oeste                         10
La Plata                           4
Mendoza                            3
Ciudad de Mendoza                  3
Cordoba                            2
Rosario                            2
Corrientes                         1
Santa Cruz                         1
San Juan                           1
Santa Fe                           1
Neuquen                            1
Tucuman                            1
Catamarca                          1
Name: nombre_zona, dtype: int64

In [47]:
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Gran Buenos Aires'),'nombre_zona'] = 3
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Capital Federal'),'nombre_zona'] = 2
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Buenos Aires (fuera de GBA)'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'GBA Oeste'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'La Plata'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Ciudad de Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Mendoza'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Catamarca'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Rosario'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Cruz'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'San Juan'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Cordoba'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Tucuman'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Neuquen'),'nombre_zona'] = 1
avisos_detalles.loc[(avisos_detalles['nombre_zona'] == 'Santa Fe'),'nombre_zona'] = 1

Pasamos a valor numérico el tipo de trabajo, según un orden jerárquico asignado por nosotros.

In [48]:
avisos_detalles['tipo_de_trabajo'].value_counts()

Full-time          22831
Part-time           1746
Teletrabajo          248
Por Horas            125
Pasantia             119
Temporario            96
Por Contrato          88
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [49]:
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Full-time'),'tipo_de_trabajo'] = 10
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Part-time'),'tipo_de_trabajo'] = 9
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Teletrabajo'),'tipo_de_trabajo'] = 8
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Horas'),'tipo_de_trabajo'] = 7
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Pasantia'),'tipo_de_trabajo'] = 6
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Temporario'),'tipo_de_trabajo'] = 5
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Por Contrato'),'tipo_de_trabajo'] = 4
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Fines de Semana'),'tipo_de_trabajo'] = 3
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Primer empleo'),'tipo_de_trabajo'] = 2
avisos_detalles.loc[(avisos_detalles['tipo_de_trabajo'] == 'Voluntario'),'tipo_de_trabajo'] = 1

Pasamos a valor numérico el nivel laboral, según un orden jerárquico asignado por nosotros.

In [50]:
avisos_detalles['nivel_laboral'].value_counts()

Senior / Semi-Senior                    16980
Junior                                   4149
Otro                                     1975
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

In [51]:
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Gerencia / Alta Gerencia / Dirección'),'nivel_laboral'] = 5
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Jefe / Supervisor / Responsable'),'nivel_laboral'] = 4
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Senior / Semi-Senior'),'nivel_laboral'] = 3
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Junior'),'nivel_laboral'] = 2
avisos_detalles.loc[(avisos_detalles['nivel_laboral'] == 'Otro'),'nivel_laboral'] = 1
avisos_detalles.loc[(avisos_detalles['nivel_laboral'].isnull()),'nivel_laboral'] = 0

In [52]:
avisos_detalles.sample(5)

,idaviso,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
14061,1112458862,<p>Estamos en búsqueda de un <strong>ANALISTA ...,3,10,3,Finanzas
8122,1112372206,<p>Nos encontramos en la búsqueda de profesion...,3,10,3,Impuestos
5566,1112362172,<p>Importante empresa radicada en la ciudad de...,3,10,1,Almacén / Depósito / Expedición
12703,1112454341,"<p>ADT Security Services, la empresa de seguri...",3,10,3,Comercial
1112,1112280752,<p> </p><p> <strong>Multinacional Automotriz ...,3,9,2,Administración


In [53]:
avisos_detalles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 0 to 25900
Data columns (total 6 columns):
idaviso            25288 non-null int64
descripcion        25288 non-null object
nombre_zona        25288 non-null object
tipo_de_trabajo    25288 non-null int64
nivel_laboral      25288 non-null int64
nombre_area        25288 non-null object
dtypes: int64(3), object(3)
memory usage: 1.4+ MB


Liberamos memoria para mejorar la performance

In [54]:
lista = 0
fecha = 0
postulaciones_2 = 0
avisos_detalles_2 = 0
postulantes_educ = 0
postulantes_gen_nac = 0

# Modelo Final

A continuación, utilizaremos los dataframes que procesamos en la sección anterior para generar el modelo final que será utilizado como set de entrenamiento por los algoritmos de ML.

Básicamente, utilizamos el dataframe de postulaciones para tener registros con postulaciones efectivas (sepostulo = 1). Luego, para generar postulaciones no efectivas (sepostulo = 0), tomamos todos los avisos para los cuales tenemos detalles y los intersectamos con una cantidad random prefijada de postulantes para los cuales tenemos información.

La siguiente función devuelve una cantidad random de ids de postulantes

In [55]:
def devolver_n_ids(n_ids):
    return np.array(postulantes['idpostulante'].sample(n_ids))

Generamos una lista de tuplas (idaviso, idpostulante) con idpostulante random para todos los avisos. Luego, llevamos esto a un dataframe y agregamos la columna 'sepostulo_y', que nos será de gran utilidad al momento de mergearlo con el dataframe de postulaciones

In [56]:
t0 = time()

ids_avisos = np.array(avisos_detalles['idaviso'])
n_ids = 250

lista_avisos_postulantes = []

for idaviso in ids_avisos:
    ids_postulantes = devolver_n_ids(n_ids)
    for idpostulante in ids_postulantes:
        tupla = (idaviso, idpostulante)
        lista_avisos_postulantes.append(tupla)
        
tf = time() - t0
print("Tiempo de ejecución:", tf)

Tiempo de ejecución: 303.11937618255615


In [57]:
df_avisos_postulantes_all = pd.DataFrame(lista_avisos_postulantes, columns=['idaviso','idpostulante'])
df_avisos_postulantes_all['sepostulo_y'] = 0

df_avisos_postulantes_all.head(10)

,idaviso,idpostulante,sepostulo_y
0,1111556097,ZOKe31,0
1,1111556097,EBZZKz,0
2,1111556097,1QrAj3E,0
3,1111556097,8Ml4DRM,0
4,1111556097,QNaYDjW,0
5,1111556097,JBX2Xar,0
6,1111556097,1bVQ2E,0
7,1111556097,dY9DbLq,0
8,1111556097,PmGreaj,0
9,1111556097,ZrzRM8,0


In [58]:
df_avisos_postulantes_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6322000 entries, 0 to 6321999
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
sepostulo_y     int64
dtypes: int64(2), object(1)
memory usage: 144.7+ MB


Mergeamos el dataframe arriba obtenido con el de postulaciones. Se realiza un outer join para quedarnos con absolutamente todas las combinaciones de idaviso e idpostulante que generamos. Además, gracias a esto, tendremos valores 0 y 1 para la columna 'sepostulo'.

In [59]:
t0 = time()

modelo = pd.merge(df_avisos_postulantes_all, postulaciones, on=['idaviso','idpostulante'], how='outer')

tf = time() - t0
print("Tiempo de ejecución:", tf)

modelo.head(5)

Tiempo de ejecución: 9.947751998901367


,idaviso,idpostulante,sepostulo_y,sepostulo_x
0,1111556097,ZOKe31,0.0,NaN
1,1111556097,EBZZKz,0.0,NaN
2,1111556097,1QrAj3E,0.0,NaN
3,1111556097,8Ml4DRM,0.0,NaN
4,1111556097,QNaYDjW,0.0,NaN


Pasamos los valores nulos a cero, para poder realizar las operaciones necesarias

In [60]:
modelo.loc[(modelo['sepostulo_x'].isnull()), 'sepostulo_x'] = 0
modelo.loc[(modelo['sepostulo_y'].isnull()), 'sepostulo_y'] = 0

modelo.head(5)

,idaviso,idpostulante,sepostulo_y,sepostulo_x
0,1111556097,ZOKe31,0.0,0.0
1,1111556097,EBZZKz,0.0,0.0
2,1111556097,1QrAj3E,0.0,0.0
3,1111556097,8Ml4DRM,0.0,0.0
4,1111556097,QNaYDjW,0.0,0.0


In [61]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12922474 entries, 0 to 12922473
Data columns (total 4 columns):
idaviso         int64
idpostulante    object
sepostulo_y     float64
sepostulo_x     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 493.0+ MB


Finalmente creamos la columna 'sepostulo'. Ésta surge de una suma lógica entre las columnas 'sepostulo_x' y 'sepostulo_y'.

In [62]:
modelo['sepostulo'] = modelo['sepostulo_x'] + modelo['sepostulo_y']
modelo.drop(columns={'sepostulo_x','sepostulo_y'}, inplace=True)

modelo.head(5)

,idaviso,idpostulante,sepostulo
0,1111556097,ZOKe31,0.0
1,1111556097,EBZZKz,0.0
2,1111556097,1QrAj3E,0.0
3,1111556097,8Ml4DRM,0.0
4,1111556097,QNaYDjW,0.0


In [63]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12922474 entries, 0 to 12922473
Data columns (total 3 columns):
idaviso         int64
idpostulante    object
sepostulo       float64
dtypes: float64(1), int64(1), object(1)
memory usage: 394.4+ MB


Como podemos ver, obtuvimos una cantidad muy pareja de postulaciones y no postulaciones

In [64]:
modelo['sepostulo'].value_counts()

1.0    6603752
0.0    6318722
Name: sepostulo, dtype: int64

In [68]:
avisos_detalles['idaviso'] = avisos_detalles['idaviso'].astype('object')
modelo['idaviso'] = modelo['idaviso'].astype('object')
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12922474 entries, 0 to 12922473
Data columns (total 3 columns):
idaviso         object
idpostulante    object
sepostulo       float64
dtypes: float64(1), object(2)
memory usage: 394.4+ MB


### Merge del modelo final con los detalles de los avisos

In [69]:
modelo = pd.merge(modelo, avisos_detalles, on=['idaviso'], how='inner')
modelo.head(4)

,idaviso,idpostulante,sepostulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,1111556097,ZOKe31,0.0,<p>Buscamos un <strong>Responsable de Producto...,3,10,3,Desarrollo de Negocios
1,1111556097,EBZZKz,0.0,<p>Buscamos un <strong>Responsable de Producto...,3,10,3,Desarrollo de Negocios
2,1111556097,1QrAj3E,0.0,<p>Buscamos un <strong>Responsable de Producto...,3,10,3,Desarrollo de Negocios
3,1111556097,8Ml4DRM,0.0,<p>Buscamos un <strong>Responsable de Producto...,3,10,3,Desarrollo de Negocios


In [70]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12506651 entries, 0 to 12506650
Data columns (total 8 columns):
idaviso            object
idpostulante       object
sepostulo          float64
descripcion        object
nombre_zona        object
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
dtypes: float64(1), int64(2), object(5)
memory usage: 858.8+ MB


Verificamos que el merge fue satisfactorio y no cuenta con valores nulos

In [71]:
(modelo['tipo_de_trabajo'].isnull().sum() == 0 , modelo['nivel_laboral'].isnull().sum() == 0)

(True, True)

### Merge del modelo final con la información de los postulantes

In [72]:
modelo = pd.merge(modelo, postulantes, on='idpostulante', how='inner')
modelo.head(4)

,idaviso,idpostulante,sepostulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,estudios,esta_estudiando,sexo,rango_edad
0,1111556097,ZOKe31,0.0,<p>Buscamos un <strong>Responsable de Producto...,3,10,3,Desarrollo de Negocios,3.0,1.0,1,4.0
1,1112017213,ZOKe31,0.0,<p>Cognizant se encuentra en la búsqueda de un...,3,10,3,Tecnologia / Sistemas,3.0,1.0,1,4.0
2,1112414508,ZOKe31,0.0,<p>Importante consultora se encuentra en la bú...,3,10,3,Ventas,3.0,1.0,1,4.0
3,1112415111,ZOKe31,1.0,"<p style=""""><strong>¿Tenes experiencia comerci...",3,10,3,Ventas,3.0,1.0,1,4.0


In [73]:
modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12252699 entries, 0 to 12252698
Data columns (total 12 columns):
idaviso            object
idpostulante       object
sepostulo          float64
descripcion        object
nombre_zona        object
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
estudios           float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
dtypes: float64(4), int64(3), object(5)
memory usage: 1.2+ GB


Verificamos que el merge fue satisfactorio y no cuenta con valores nulos

In [74]:
(modelo['estudios'].isnull().sum() == 0 , modelo['rango_edad'].isnull().sum() == 0)

(True, True)

Liberamos memoria para mejorar la performance

In [75]:
df_avisos_postulantes_all = 0
postulantes = 0
postulaciones = 0
avisos_detalles = 0

Analizamos la información del modelo final que será utilizado por los algoritmos de Machine Learning.

In [76]:
modelo.rename(columns={'estudios':'nivel_estudios'},inplace=True)

modelo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12252699 entries, 0 to 12252698
Data columns (total 12 columns):
idaviso            object
idpostulante       object
sepostulo          float64
descripcion        object
nombre_zona        object
tipo_de_trabajo    int64
nivel_laboral      int64
nombre_area        object
nivel_estudios     float64
esta_estudiando    float64
sexo               int64
rango_edad         float64
dtypes: float64(4), int64(3), object(5)
memory usage: 1.2+ GB


Analizamos la cantidad de postulaciones y no postulaciones con las que cuenta el modelo

In [77]:
modelo['sepostulo'].value_counts()

0.0    6318722
1.0    5933977
Name: sepostulo, dtype: int64

Considerando la gran cantidad de registros que tiene el dataframe, decidimos tomar una porción aleatoria y equitativa de postulaciones y no postulaciones

In [78]:
no = modelo['sepostulo'] == 0
si = modelo['sepostulo'] == 1

modelo_no = modelo[no]
modelo_si = modelo[si]

In [79]:
n_samples = 500000

modelo_no = modelo_no.sample(n_samples)
modelo_si = modelo_si.sample(n_samples)

modelo_final = pd.concat([modelo_no, modelo_si])

modelo_final.sample(5)

,idaviso,idpostulante,sepostulo,descripcion,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area,nivel_estudios,esta_estudiando,sexo,rango_edad
5821316,1112452595,96XDVMY,1.0,"<p>Netmart ., Somos una empresa con más de 5 a...",3,10,3,Ventas,3.0,0.0,2,3.0
6563093,1112437163,1QrxVOb,0.0,<p>Importante empresa de servicios se encuentr...,3,10,3,Administración,4.0,1.0,2,2.0
5892427,1112212678,mzdPLVO,1.0,<p><strong>Adecco Office</strong> está especia...,3,10,3,Abastecimiento,4.0,0.0,2,4.0
9611409,1111417889,NB25jz,0.0,<p>Importante empresa se encuentra en la búsqu...,3,10,3,Calidad,3.0,1.0,2,4.0
3860383,1112353973,qe26xVk,0.0,"<p><span style=""text-decoration: underline;""><...",3,10,2,Planeamiento,2.0,0.0,2,1.0


In [80]:
modelo_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 9830236 to 5981004
Data columns (total 12 columns):
idaviso            1000000 non-null object
idpostulante       1000000 non-null object
sepostulo          1000000 non-null float64
descripcion        1000000 non-null object
nombre_zona        1000000 non-null object
tipo_de_trabajo    1000000 non-null int64
nivel_laboral      1000000 non-null int64
nombre_area        1000000 non-null object
nivel_estudios     1000000 non-null float64
esta_estudiando    1000000 non-null float64
sexo               1000000 non-null int64
rango_edad         1000000 non-null float64
dtypes: float64(4), int64(3), object(5)
memory usage: 99.2+ MB


Liberamos memoria para mejorar la performance

In [81]:
modelo_no = 0
modelo_si = 0

Reordenamos las columnas para facilitar el manejo del dataframe cuando apliquemos los algoritmos de ML

In [83]:
modelo_final = modelo_final[['idaviso','idpostulante','rango_edad','sexo',\
                             'nivel_estudios','esta_estudiando','descripcion','tipo_de_trabajo',\
                             'nivel_laboral','nombre_zona','nombre_area','sepostulo']]

modelo_final.sample(5)

,idaviso,idpostulante,rango_edad,sexo,nivel_estudios,esta_estudiando,descripcion,tipo_de_trabajo,nivel_laboral,nombre_zona,nombre_area,sepostulo
7674663,1112442348,YjrpVzP,1.0,1,3.0,0.0,"<p style="""">Empresa multinacional de Servicios...",9,3,3,Atención al Cliente,1.0
268447,1112363604,2zP1zZa,1.0,1,2.0,1.0,<p>GANÁ SIN MANEJAR ! ACTIVIDAD INDEPENDIENTE<...,8,3,3,Ventas,1.0
8029057,1112455267,VNrER0R,1.0,1,3.0,1.0,<p>Somos una compañía multinacional líder en s...,10,3,3,Tecnologia / Sistemas,0.0
9836850,1112326604,0zrR9AM,1.0,1,3.0,1.0,<p>Grupo empresario con cobertura nacional y o...,10,3,3,Administración,0.0
11177802,1112309768,ZX8QPV,3.0,1,4.0,1.0,"<p><span style="""">Recepcionista - Telefonista ...",5,3,3,Recepcionista,0.0


Transformamos las variables categóricas del tipo entero, objeto o flotante a categóricas propiamente dichas. Esto mejora la performance notablemente.

In [84]:
modelo_final['rango_edad'] = modelo_final['rango_edad'].astype('category')
modelo_final['sexo'] = modelo_final['sexo'].astype('category') 
modelo_final['nivel_estudios'] = modelo_final['nivel_estudios'].astype('category')
modelo_final['esta_estudiando'] = modelo_final['esta_estudiando'].astype('category')
modelo_final['tipo_de_trabajo'] = modelo_final['tipo_de_trabajo'].astype('category')
modelo_final['nivel_laboral'] = modelo_final['nivel_laboral'].astype('category')
modelo_final['nombre_zona'] = modelo_final['nombre_zona'].astype('category')
modelo_final['sepostulo'] = modelo_final['sepostulo'].astype('category')

In [85]:
modelo_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 9830236 to 5981004
Data columns (total 12 columns):
idaviso            1000000 non-null object
idpostulante       1000000 non-null object
rango_edad         1000000 non-null category
sexo               1000000 non-null category
nivel_estudios     1000000 non-null category
esta_estudiando    1000000 non-null category
descripcion        1000000 non-null object
tipo_de_trabajo    1000000 non-null category
nivel_laboral      1000000 non-null category
nombre_zona        1000000 non-null category
nombre_area        1000000 non-null object
sepostulo          1000000 non-null category
dtypes: category(8), object(4)
memory usage: 45.8+ MB


Exportamos el modelo final a un CSV file, para desentendernos de todas las operaciones costosas de dataframes a partir de este punto, y poder tener el set de entrenamiento ya listo para los algoritmos de Machine Learning.

In [86]:
modelo_final.to_csv("modelo_final.csv", encoding = "utf-8", index = False)